# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -U adapter-transformers
!pip install datasets

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/rct-20k-tlm")
dataset.num_rows

Using custom data configuration yxchar___rct-20k-tlm-5e3653a5bfe528ee
Reusing dataset csv (C:\Users\Junfe\.cache\huggingface\datasets\csv\yxchar___rct-20k-tlm-5e3653a5bfe528ee\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 180040, 'test': 30135, 'validation': 30212}

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 180040
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 30135
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 30212
    })
})

Every dataset sample has an input text and a binary label:

In [4]:
import numpy as np
np.unique(dataset['train']['label'])

array([0, 1, 2, 3, 4])

In [5]:
np.max([len(x['text']) for x in dataset['train']])

1454

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 180040
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 30135
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'id', 'label'],
        num_rows: 30212
    })
})

In [7]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\csv\yxchar___rct-20k-tlm-5e3653a5bfe528ee\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-0e76923bcceb0050.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\csv\yxchar___rct-20k-tlm-5e3653a5bfe528ee\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-a471b34f248fad5b.arrow


  0%|          | 0/31 [00:00<?, ?ba/s]

C:\Users\Junfe\AppData\Local\Temp/ipykernel_38364/1657904570.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  dataset.rename_column_("label", "labels")


Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [8]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=5,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [9]:
# Add a new adapter
model.add_adapter("rct-20k")
# Add a matching classification head
model.add_classification_head(
    "rct-20k",
    num_labels=5,
    # id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("rct-20k")

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [10]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [11]:
trainer.train()

***** Running training *****
  Num examples = 180040
  Num Epochs = 6
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 90024


Step,Training Loss
200,1.052500
400,0.642500
600,0.583000
800,0.552200
1000,0.527300
1200,0.528400
1400,0.482000
1600,0.467900
1800,0.465900
2000,0.486000


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-500\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-1000
Configuration saved in ./training_output\checkpoint-1000\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-1000\rct-20k\pytorch_adapter.bin
Configuration saved in ./trainin

Configuration saved in ./training_output\checkpoint-6000\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-6000\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-6000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-6000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-6000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-6000\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-6500
Configuration saved in ./training_output\checkpoint-6500\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-6500\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-6500\rct-20k\head_config.json
Mo

Configuration saved in ./training_output\checkpoint-11500\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-11500\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-11500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-11500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-11500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-11500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-11500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-11500\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-12000
Configuration saved in ./training_output\checkpoint-12000\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-12000\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-12000\rct-20k\head_co

...

Module weights saved in ./training_output\checkpoint-71000\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-71500
Configuration saved in ./training_output\checkpoint-71500\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-71500\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-71500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-71500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-71500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-71500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-71500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-71500\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-72000
Configuration saved in ./training_output\checkpoint-72000\rct-20k\adapter_config.json
Module w

Configuration saved in ./training_output\checkpoint-76500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-76500\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-77000
Configuration saved in ./training_output\checkpoint-77000\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-77000\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-77000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-77000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-77000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-77000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-77000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-77000\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-77500
Configurati

Module weights saved in ./training_output\checkpoint-82000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-82000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-82000\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-82500
Configuration saved in ./training_output\checkpoint-82500\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-82500\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-82500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-82500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-82500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-82500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-82500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-82500\rct-20k\pytorc

Configuration saved in ./training_output\checkpoint-87500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-87500\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-87500\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-87500\rct-20k\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-88000
Configuration saved in ./training_output\checkpoint-88000\rct-20k\adapter_config.json
Module weights saved in ./training_output\checkpoint-88000\rct-20k\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-88000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-88000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-88000\rct-20k\head_config.json
Module weights saved in ./training_output\checkpoint-88000\rct-20k\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-88000\rct-20k\head_co

TrainOutput(global_step=90024, training_loss=0.359972365843416, metrics={'train_runtime': 18285.6082, 'train_samples_per_second': 59.076, 'train_steps_per_second': 4.923, 'total_flos': 2.8915910889578496e+17, 'train_loss': 0.359972365843416, 'epoch': 6.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [12]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30212
  Batch size = 12


{'eval_loss': 0.34947219491004944,
 'eval_acc': 0.8803455580564015,
 'eval_runtime': 227.7264,
 'eval_samples_per_second': 132.668,
 'eval_steps_per_second': 11.057,
 'epoch': 6.0}

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

In [13]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': 'LABEL_3', 'score': 0.9442794919013977}]

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [16]:
model.save_adapter("./final_adapter", "rct-20k")

!ls -lh final_adapter

Configuration saved in ./final_adapter\adapter_config.json
Module weights saved in ./final_adapter\pytorch_adapter.bin
Configuration saved in ./final_adapter\head_config.json
Module weights saved in ./final_adapter\pytorch_model_head.bin
'ls' is not recognized as an internal or external command,
operable program or batch file.


**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.